## Real or Not? NLP with Disaster Tweets - Project 2 - Sam's part

Big-Scale Analytics - Project 2 - Team Rolex
> Samuel Lew, Alexandre Lang, Samy Bouzerda, Alix Muller

## Load dataset

In [2]:
import pandas as pd

In [46]:
df = pd.read_csv ('../data/train.csv')
df = df[['id', 'text', 'target']]
df.iloc[:10]

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
5,8,#RockyFire Update => California Hwy. 20 closed...,1
6,10,#flood #disaster Heavy rain causes flash flood...,1
7,13,I'm on top of the hill and I can see a fire in...,1
8,14,There's an emergency evacuation happening now ...,1
9,15,I'm afraid that the tornado is coming to our a...,1


In [47]:
index = df.index
columns = df.columns
values = df.values

In [56]:
df_yes = df.query('target == 1')
yes_count = 0
for row in df_yes.iterrows():
    yes_count += 1
print(f'there are {yes_count} tweets about disasters')

df_no = df.query('target == 0')
no_count = 0
for row in df_no.iterrows():
    no_count += 1
print(f'there are {no_count} tweets that are not about disasters')

there are 3271 tweets about disasters
there are 4342 tweets that are not about disasters


## Process text

In [59]:
!pip install spacy
!python -m spacy download en

import spacy
from spacy.lang.en import English

You should consider upgrading via the '/Users/Samuel/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/Samuel/anaconda3/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/Samuel/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/Users/Samuel/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [63]:
nlp = English()
sbd = nlp.create_pipe('sentencizer')
nlp.add_pipe(sbd)

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [65]:
for tweet in df['text']:
    tweet = nlp(tweet)
    filt_tweet = []
    for word in tweet:
        if word.is_stop==False:
            filt_tweet.append(word)